**Loading the dataset from google drive**

In [1]:
from google.colab import drive
#drive.mount('/content/drive')

In [2]:
ls -l

total 1084232
-rw-r--r-- 1 root root 800419647 Sep 20  2019 blogtext.csv
-rw------- 1 root root 303646744 Sep 25 05:26 data.zip
drwx------ 4 root root      4096 Sep 25 05:25 drive/
-rw-r--r-- 1 root root   6167444 Sep 25 08:57 newdf.csv
drwxr-xr-x 1 root root      4096 Sep 16 16:29 sample_data/


In [3]:
!unzip data.zip

Archive:  data.zip
replace blogtext.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: blogtext.csv            


In [4]:
import pandas as pd
import nltk
import gensim
import seaborn as sns
import numpy as np
import re
import unicodedata
nltk.download('punkt')
nltk.download('wordnet')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df = pd.read_csv("blogtext.csv")

In [6]:
df = df.head(10000)

**Exploring the data**

In [7]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10000 non-null  int64 
 1   gender  10000 non-null  object
 2   age     10000 non-null  int64 
 3   topic   10000 non-null  object
 4   sign    10000 non-null  object
 5   date    10000 non-null  object
 6   text    10000 non-null  object
dtypes: int64(2), object(5)
memory usage: 547.0+ KB


In [9]:
df.shape

(10000, 7)

In [10]:
df.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [11]:
print(df.text[2])

           In het kader van kernfusie op aarde:  MAAK JE EIGEN WATERSTOFBOM   How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia  Original file dated 12th November 1990. Seemed to be a transcript of a 'Seven Days' article. Poorly formatted and corrupted. I have added the text between 'examine under a microscope' and 'malleable, like gold,' as it was missing. If anyone has the full text, please distribute. I am not responsible for the accuracy of this information. Converted to HTML by Dionisio@InfiNet.com 11/13/98. (Did a little spell-checking and some minor edits too.) Stolen from  urlLink http://my.ohio.voyager.net/~dionisio/fun/m...own-h-bomb.html  and reformatted the HTML. It now validates to XHTML 1.0 Strict. How to Build an H-Bomb Making and owning an H-bomb is the kind of challenge real Americans seek. Who wants to b

In [12]:
df.gender.unique()

array(['male', 'female'], dtype=object)

In [13]:
df.topic.unique()

array(['Student', 'InvestmentBanking', 'indUnk', 'Non-Profit', 'Banking',
       'Education', 'Engineering', 'Science', 'Communications-Media',
       'BusinessServices', 'Sports-Recreation', 'Arts', 'Internet',
       'Museums-Libraries', 'Accounting', 'Technology', 'Law',
       'Consulting', 'Automotive', 'Religion', 'Fashion', 'Publishing',
       'Marketing', 'LawEnforcement-Security', 'HumanResources',
       'Telecommunications'], dtype=object)

In [14]:
df.sign.unique()

array(['Leo', 'Aquarius', 'Aries', 'Capricorn', 'Gemini', 'Cancer',
       'Sagittarius', 'Scorpio', 'Libra', 'Virgo', 'Taurus', 'Pisces'],
      dtype=object)

In [15]:
df.age.unique()

array([15, 33, 14, 25, 17, 23, 37, 26, 24, 27, 45, 34, 41, 44, 16, 39, 35,
       36, 46, 42, 13, 38, 43, 40])

In [16]:
df['age']= df['age'].astype(str)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10000 non-null  int64 
 1   gender  10000 non-null  object
 2   age     10000 non-null  object
 3   topic   10000 non-null  object
 4   sign    10000 non-null  object
 5   date    10000 non-null  object
 6   text    10000 non-null  object
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [18]:
df.head(0)

,id,gender,age,topic,sign,date,text


In [19]:
df.drop(['id', 'date'], axis=1, inplace=True)

In [20]:
df.head(0)

,gender,age,topic,sign,text




1.   Remove unwanted characters
2.   Convert text to lovercase
3.    Remove unwanted spaces



In [21]:
df['gender'] = df['gender'].apply(lambda s: s.lower())
df['topic'] = df['topic'].apply(lambda s: s.lower())
df['sign'] = df['sign'].apply(lambda s: s.lower())
df['text'] = df['text'].apply(lambda s: s.lower())

In [22]:
df['gender'] = df['gender'].apply(lambda s: re.sub('[^a-z ]','',s))
df['topic'] = df['topic'].apply(lambda s: re.sub('[^a-z ]','',s))
df['sign'] = df['sign'].apply(lambda s: re.sub('[^a-z ]','',s))
df['text'] = df['text'].apply(lambda s: re.sub('[^a-z ]','',s))

In [23]:
df['gender'] = df['gender'].apply(lambda s: s.strip())
df['topic'] = df['topic'].apply(lambda s: s.strip())
df['sign'] = df['sign'].apply(lambda s: s.strip())
df['text'] = df['text'].apply(lambda s: s.strip())

In [24]:
df.head()

,gender,age,topic,sign,text
0,male,15,student,leo,info has been found pages and mb of pdf fil...
1,male,15,student,leo,these are the team members drewes van der la...
2,male,15,student,leo,in het kader van kernfusie op aarde maak je e...
3,male,15,student,leo,testing testing
4,male,33,investmentbanking,aquarius,thanks to yahoos toolbar i can now capture the...


In [25]:
print(df.text[2])

in het kader van kernfusie op aarde  maak je eigen waterstofbom   how to build an hbomb from ascotttartarusuwaeduau andrew scott newsgroups rechumor subject how to build an hbomb humorous date  feb   gmt organization the university of western australia  original file dated th november  seemed to be a transcript of a seven days article poorly formatted and corrupted i have added the text between examine under a microscope and malleable like gold as it was missing if anyone has the full text please distribute i am not responsible for the accuracy of this information converted to html by dionisioinfinetcom  did a little spellchecking and some minor edits too stolen from  urllink httpmyohiovoyagernetdionisiofunmownhbombhtml  and reformatted the html it now validates to xhtml  strict how to build an hbomb making and owning an hbomb is the kind of challenge real americans seek who wants to be a passive victim of nuclear war when with a little effort you can be an active participant bomb shel

In [26]:
print(df.sign.value_counts())

aries          4198
sagittarius    1097
scorpio         971
taurus          812
aquarius        571
cancer          504
libra           491
pisces          454
leo             301
virgo           236
capricorn       215
gemini          150
Name: sign, dtype: int64


**Removing stopwords**

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
stopword_list = nltk.corpus.stopwords.words('english')

In [29]:
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopword_list)]))

In [30]:
df.head()

,gender,age,topic,sign,text
0,male,15,student,leo,info found pages mb pdf files wait untill team...
1,male,15,student,leo,team members drewes van der laag urllink mail ...
2,male,15,student,leo,het kader van kernfusie op aarde maak je eigen...
3,male,15,student,leo,testing testing
4,male,33,investmentbanking,aquarius,thanks yahoos toolbar capture urls popupswhich...


In [31]:
print(df.text[2])

het kader van kernfusie op aarde maak je eigen waterstofbom build hbomb ascotttartarusuwaeduau andrew scott newsgroups rechumor subject build hbomb humorous date feb gmt organization university western australia original file dated th november seemed transcript seven days article poorly formatted corrupted added text examine microscope malleable like gold missing anyone full text please distribute responsible accuracy information converted html dionisioinfinetcom little spellchecking minor edits stolen urllink httpmyohiovoyagernetdionisiofunmownhbombhtml reformatted html validates xhtml strict build hbomb making owning hbomb kind challenge real americans seek wants passive victim nuclear war little effort active participant bomb shelters losers wants huddle together underground eating canned spam winners want push button making hbomb big step nuclear assertiveness training called taking charge sure youll enjoy risks heady thrill playing nuclear chicken introduction feds clamped progres

In [32]:
df['labels'] = df[df.columns[:4]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)


In [33]:
df.head()

,gender,age,topic,sign,text,labels
0,male,15,student,leo,info found pages mb pdf files wait untill team...,"male,15,student,leo"
1,male,15,student,leo,team members drewes van der laag urllink mail ...,"male,15,student,leo"
2,male,15,student,leo,het kader van kernfusie op aarde maak je eigen...,"male,15,student,leo"
3,male,15,student,leo,testing testing,"male,15,student,leo"
4,male,33,investmentbanking,aquarius,thanks yahoos toolbar capture urls popupswhich...,"male,33,investmentbanking,aquarius"


In [34]:
df.tail(5)

,gender,age,topic,sign,text,labels
9995,female,25,indunk,pisces,take home forever may rest sleep arms forgotte...,"female,25,indunk,pisces"
9996,female,25,indunk,pisces,seductive secretness behind doors warning neve...,"female,25,indunk,pisces"
9997,female,25,indunk,pisces,kind need holding hand petting hair cry bring ...,"female,25,indunk,pisces"
9998,female,25,indunk,pisces,blurry outside sounds people mingle pass darkn...,"female,25,indunk,pisces"
9999,female,25,indunk,pisces,body feels broken mind rejoices thought warmth...,"female,25,indunk,pisces"


In [35]:
df.shape

(10000, 6)

In [36]:
df.to_csv('newdf.csv')

**Creating labels and feature from the dataframe**

In [37]:
X = df['text']
y = df['labels']

In [38]:
print(X.shape)
print(y.shape)

(10000,)
(10000,)


**Splitting the feature and labels into train and test**


In [39]:
from sklearn.model_selection import train_test_split

In [40]:
test_size = 0.25 # taking 75:25 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [41]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7500,)
(7500,)
(2500,)
(2500,)


**Initialising count vectorizer**

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(ngram_range=(1,2), stop_words='english', min_df=2)

**Transforing train and test into Count vectorizer**

In [43]:
X_train = cvect.fit_transform(X_train)

In [44]:
X_test= cvect.transform(X_test)

In [45]:
cvect.get_feature_names()[:5]

['aa', 'aaa', 'aaron', 'aarons', 'ab']

In [46]:
len(cvect.vocabulary_)

64348

In [47]:
print(X_train.shape)
print(X_test.shape)


(7500, 64348)
(2500, 64348)


**Creating Document term matrix and printing**

In [48]:
dtm = pd.DataFrame(X_train.toarray(), columns = cvect.get_feature_names())

In [49]:
dtm

,aa,aaa,aaron,aarons,ab,aba,aback,aback bit,aback coz,abandon,abandoned,abandoning,abandons,abbey,abbots,abbott,abby,abc,abc news,abcnewscom,abcs,abdomen,abdul,abercrombie,abhor,abilities,ability,ability create,ability generate,ability make,ability stand,ability use,able,able afford,able avoid,able blow,able car,able catch,able come,able connect,...,zeldman,zellweger,zelo,zen,zenith,zeppelin,zero,zest,zhadoxero,zhang,zhao,ziggy,ziggy stardust,zillion,zimbabwe,zion,zip,zip code,zipper,zipping,zmx,zmx dont,zmx hate,zmx thing,zmx think,zodiac,zodiac forecasts,zodiac sign,zombie,zone,zone natural,zones,zoo,zoo animal,zoolander,zoom,zoos,zs,zzzs,zzzzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7496,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Creating dictionary**

In [50]:
dfT = df[['gender', 'age', 'topic', 'sign']]

In [51]:
keys=[] 
values=[] 

for i in range(dfT.shape[1]): # iterate through all the colummns        
    for j in range(dfT.iloc[:,i].value_counts().shape[0]): # iterate through all the rows of value_counts of that column
        keys.append(dfT.iloc[:,i].value_counts().index[j])         
        values.append(dfT.iloc[:,i].value_counts().iloc[j])

In [52]:
dictionary = dict(zip(keys,values))

In [53]:
dictionary

{'13': 42,
 '14': 212,
 '15': 602,
 '16': 440,
 '17': 1185,
 '23': 253,
 '24': 655,
 '25': 386,
 '26': 234,
 '27': 1054,
 '33': 136,
 '34': 553,
 '35': 2315,
 '36': 1708,
 '37': 33,
 '38': 46,
 '39': 79,
 '40': 1,
 '41': 20,
 '42': 14,
 '43': 6,
 '44': 3,
 '45': 16,
 '46': 7,
 'accounting': 4,
 'aquarius': 571,
 'aries': 4198,
 'arts': 45,
 'automotive': 14,
 'banking': 16,
 'businessservices': 91,
 'cancer': 504,
 'capricorn': 215,
 'communicationsmedia': 99,
 'consulting': 21,
 'education': 270,
 'engineering': 127,
 'fashion': 1622,
 'female': 4084,
 'gemini': 150,
 'humanresources': 2,
 'indunk': 3287,
 'internet': 118,
 'investmentbanking': 70,
 'law': 11,
 'lawenforcementsecurity': 10,
 'leo': 301,
 'libra': 491,
 'male': 5916,
 'marketing': 156,
 'museumslibraries': 17,
 'nonprofit': 71,
 'pisces': 454,
 'publishing': 4,
 'religion': 9,
 'sagittarius': 1097,
 'science': 63,
 'scorpio': 971,
 'sportsrecreation': 80,
 'student': 1137,
 'taurus': 812,
 'technology': 2654,
 'telecom

**Initialising Multilabel Binarizer** 

In [54]:
from sklearn.preprocessing import MultiLabelBinarizer

In [55]:
y_train

4881    male,25,businessservices,sagittarius
89                    female,14,indunk,aries
845              male,17,student,sagittarius
8338        female,25,technology,sagittarius
9947              female,16,indunk,capricorn
                        ...                 
919       male,17,sportsrecreation,capricorn
4307            female,34,indunk,sagittarius
5699                 female,27,indunk,taurus
537             female,27,education,aquarius
9412                   male,16,indunk,cancer
Name: labels, Length: 7500, dtype: object

In [56]:
# transform to dictionary
y_train = [set(i.split(',')) for i in y_train]
y_test = [set(i.split(',')) for i in y_test]

In [57]:
y_train[0]

{'25', 'businessservices', 'male', 'sagittarius'}

In [58]:
mlb = MultiLabelBinarizer()

In [59]:
y_train = mlb.fit_transform(y_train)

In [60]:
y_test = mlb.transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['40'] will be ignored
  .format(sorted(unknown, key=str)))


In [61]:
y_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [62]:
mlb.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '41', '42', '43', '44', '45',
       '46', 'accounting', 'aquarius', 'aries', 'arts', 'automotive',
       'banking', 'businessservices', 'cancer', 'capricorn',
       'communicationsmedia', 'consulting', 'education', 'engineering',
       'fashion', 'female', 'gemini', 'humanresources', 'indunk',
       'internet', 'investmentbanking', 'law', 'lawenforcementsecurity',
       'leo', 'libra', 'male', 'marketing', 'museumslibraries',
       'nonprofit', 'pisces', 'publishing', 'religion', 'sagittarius',
       'science', 'scorpio', 'sportsrecreation', 'student', 'taurus',
       'technology', 'telecommunications', 'virgo'], dtype=object)

In [63]:
print(y_train.shape)
print(y_train.shape)

(7500, 63)
(7500, 63)


In [64]:
y_train[156]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [65]:
y_test[786]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

**Using Logistic Regression as a classifier with OneVsRest Classifier approach**

In [66]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [67]:
clf = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(clf)

In [68]:
history = clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


**Prediciting Values**

In [69]:
y_logic_predict = clf.predict(X_test)

In [70]:
y_logic_predict[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [71]:
y1 = mlb.inverse_transform(y_logic_predict)

In [72]:
y1[:10]

[('35', 'aries', 'male', 'technology'),
 ('female', 'indunk'),
 ('33', 'investmentbanking', 'male'),
 ('aries', 'male'),
 ('35', 'aries', 'male', 'technology'),
 ('male',),
 ('engineering', 'female'),
 ('35', 'aries', 'male', 'technology'),
 ('35', 'aries', 'male', 'technology'),
 ('male',)]

In [73]:
from sklearn import metrics
model_score = clf.score(X_test, y_test)

In [74]:
print(model_score)
print(metrics.classification_report(y_test, y_logic_predict))

0.3172
              precision    recall  f1-score   support

           0       0.88      0.54      0.67        13
           1       0.78      0.12      0.20        60
           2       0.66      0.30      0.41       148
           3       0.56      0.19      0.28       116
           4       0.70      0.31      0.43       293
           5       0.22      0.04      0.06        54
           6       0.53      0.19      0.28       146
           7       0.68      0.11      0.19       120
           8       0.67      0.03      0.06        62
           9       0.75      0.38      0.50       244
          10       0.89      0.42      0.58        40
          11       0.98      0.72      0.83       131
          12       0.72      0.64      0.67       587
          13       0.91      0.56      0.70       433
          14       0.00      0.00      0.00         7
          15       0.71      0.45      0.56        11
          16       0.40      0.09      0.15        22
          17       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def print_evaluation_scores(y_test, y_logic_predict):
    print('Accuracy score: ', accuracy_score(y_test, y_logic_predict))
    print('F1 score: ', f1_score(y_test, y_logic_predict, average='micro'))
    print('Average precision score: ', average_precision_score(y_test, y_logic_predict, average='micro'))
    print('Average recall score: ', recall_score(y_test, y_logic_predict, average='micro'))

In [76]:
print_evaluation_scores(y_test, y_logic_predict)

Accuracy score:  0.3172
F1 score:  0.6413138431075651
Average precision score:  0.45126914679835445
Average recall score:  0.5506550655065506


## Trying Multinomial NB Classification

In [77]:
from sklearn.naive_bayes import MultinomialNB

In [78]:
NB_model = MultinomialNB()

In [79]:
NB_model = OneVsRestClassifier(NB_model)

In [80]:
history = NB_model.fit(X_train, y_train)

In [81]:
y_logic_predict_NB = NB_model.predict(X_test)

In [82]:
y_logic_predict_NB[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [83]:
from sklearn import metrics
model_score_NB = NB_model.score(X_test, y_test)

In [84]:
print(model_score_NB)
print(metrics.classification_report(y_test, y_logic_predict_NB))

0.19
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.80      0.07      0.12        60
           2       0.63      0.13      0.21       148
           3       0.80      0.07      0.13       116
           4       0.75      0.33      0.46       293
           5       0.50      0.02      0.04        54
           6       0.61      0.08      0.13       146
           7       0.00      0.00      0.00       120
           8       1.00      0.03      0.06        62
           9       0.76      0.30      0.42       244
          10       0.33      0.03      0.05        40
          11       0.94      0.23      0.37       131
          12       0.80      0.28      0.42       587
          13       0.91      0.52      0.66       433
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0.00        11
          16       1.00      0.05      0.09        22
          17       0.0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
print_evaluation_scores(y_test, y_logic_predict_NB)

Accuracy score:  0.19
F1 score:  0.5629553879585837
Average precision score:  0.37901868672536004
Average recall score:  0.44044404440444046


## Now using TF- IDF 

**Initialising TF-IDF vectorizer**

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)

**Transforming train and test into TF-IDF vectorizer**

In [87]:
test_size = 0.25 # taking 75:25 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [88]:
X_train = tfidf.fit_transform(X_train)

In [89]:
X_test = tfidf.transform(X_test)

In [90]:
dtm_tfidf = pd.DataFrame(X_train.toarray(), columns = tfidf.get_feature_names())

In [91]:
dtm_tfidf

,aa,aaa,aaron,aarons,ab,aba,aback,aback bit,aback coz,abandon,abandoned,abandoning,abandons,abbey,abbots,abbott,abby,abc,abc news,abcnewscom,abcs,abdomen,abdul,abercrombie,abhor,abilities,ability,ability create,ability generate,ability make,ability see,ability stand,ability use,able,able afford,able anything,able avoid,able blow,able call,able catch,...,zeldman,zellweger,zelo,zen,zenith,zeppelin,zero,zest,zhadoxero,zhang,zhao,ziggy,ziggy stardust,zillion,zimbabwe,zion,zip,zip code,zipper,zipping,zmx,zmx dont,zmx hate,zmx thing,zmx think,zodiac,zodiac forecasts,zodiac sign,zombie,zone,zone natural,zones,zoo,zoo animal,zoolander,zoom,zoos,zs,zzzs,zzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
# transform to dictionary
y_train = [set(i.split(',')) for i in y_train]
y_test = [set(i.split(',')) for i in y_test]

In [93]:
y_train = mlb.fit_transform(y_train)

In [94]:
y_test = mlb.transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['40'] will be ignored
  .format(sorted(unknown, key=str)))


In [95]:
history = clf.fit(X_train, y_train)

In [96]:
y_logic_predict = clf.predict(X_test)

In [97]:
y_logic_predict[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [98]:
y1 = mlb.inverse_transform(y_logic_predict)

In [99]:
y1[:10]

[('aries', 'male'),
 ('female',),
 ('male',),
 ('aries', 'male'),
 ('aries', 'male'),
 ('female',),
 ('female',),
 ('aries', 'male'),
 ('aries', 'male'),
 ('35', 'aries', 'male', 'technology')]

In [100]:
from sklearn import metrics
model_score = clf.score(X_test, y_test)

In [101]:
print(model_score)
print(metrics.classification_report(y_test, y_logic_predict))

0.1044
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.00      0.00      0.00        60
           2       1.00      0.01      0.01       148
           3       1.00      0.01      0.02       116
           4       0.85      0.04      0.07       293
           5       0.00      0.00      0.00        54
           6       1.00      0.01      0.03       146
           7       0.00      0.00      0.00       120
           8       0.00      0.00      0.00        62
           9       0.66      0.08      0.14       244
          10       0.00      0.00      0.00        40
          11       1.00      0.11      0.21       131
          12       0.96      0.24      0.39       587
          13       0.98      0.26      0.41       433
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0.00        11
          16       0.00      0.00      0.00        22
          17       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [102]:
print_evaluation_scores(y_test, y_logic_predict)

Accuracy score:  0.1044
F1 score:  0.49616617872577723
Average precision score:  0.33230245628657645
Average recall score:  0.35593559355935595
